In [4]:
# Plot top words per topic

def plot_top_words(model, feature_names, n_top_words = 7):
  fig, axes = plt.subplots(1, 5, figsize = (30, 10), sharex = True)
  axes = axes.flatten()
  for topic_idx, topic in enumerate(model.components_):
    top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
    top_features = [feature_names[i] for i in top_features_ind]
    weights = topic[top_features_ind]
    
    ax = axes[topic_idx]
    ax.barh(top_features, weights, height = 0.7)
    ax.set_title(f'Topic {topic_idx + 1}', fontdict = {'fontsize': 30})
    ax.invert_yaxis()
    ax.tick_params(axis = 'both', which = 'major', labelsize = 20)
    for i in 'top right left'.split():
      ax.spines[i].set_visible(False)
    fig.suptitle("LDA", fontsize = 40)
  
  plt.subplots_adjust(top = 0.90, bottom = 0.05, wspace = 0.90, hspace = 0.3)
  plt.show()

  
  
  
# Estimate model accuracy:
def est_accuracy(topic_vocab=topic_vocab):
  new_sentences = lda_modeller.vectorizer.transform(df['text'])
  topic_distributions = lda.transform(new_sentences)
  most_probable_topics = topic_distributions.argmax(axis=1)
  preds = [topic_vocab[num_topic] for num_topic in most_probable_topics]
  df['preds'] = preds
  acc = sum(df['labels'] == df['preds']) / len(df)
  print(f'Supposed model accuracy: {acc}')
  
  
  
  
# Sample topic
def sample_topic():
  

NameError: name 'topic_vocab' is not defined